![](https://i.imgur.com/eHJKbYb.png)

![](https://i.imgur.com/9w3267z.png)

Kernel 작성 시점의 Titanic competition 랭크 및 Score입니다. 스코어는 competition 종료 후 제출하였기 때문에 최종 랭크와는 관계가 없습니다.

본 Kernel은 참가하신 다수의 분들이 겪으신 80% 초 ~ 중반 구간의 장벽을 어떻게 넘었는지에 대한 방법을 공유하기 위한 목적으로 작성하였습니다. 먼저 제가 작성한 방법 이외에도 **위의 구간을 넘는 다른 방법들이 존재함**을 분명히 밝힙니다. 저 또한 해당 구간을 극복하기 위해 다양한 방법을 고민하고 시도하였고, 누군가에게는 제가 한 고민이 도움이 될 수 있을것 같아 Kernel을 작성하게 되었습니다. 

1. 정확도를 올리기 위한 방법 중 일부를 공유하는 것이기 때문에 해당 Kernel에서는 위의 스코어까지 도달하는 직접적인 방법-code-은 제시하지 않습니다.

2. 데이터에 대한 시각화는 선행하셨을 것이라고 판단하고 생략하였습니다.

3. Genetic Programming에 대한 내용은 다루지 않습니다.

4. 본 Kernel의 Gender Model은 Kaggle Titanic에 있는 Kerenl을 베이스로 발전시킨 것임을 밝힙니다.
https://www.kaggle.com/cdeotte/titantic-mega-model-0-84210




In [16]:
import pandas as pd
import numpy as np
import re
# from xgboost import XGBClassifier
from sklearn import preprocessing, model_selection
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None

train = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/train.csv')
test = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/test.csv')

print(train.info())
train.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
742,743,1,1,"Ryerson, Miss. Susan Parker ""Suzette""",female,21.0,2,2,PC 17608,262.375,B57 B59 B63 B66,C
173,174,0,3,"Sivola, Mr. Antti Wilhelm",male,21.0,0,0,STON/O 2. 3101280,7.925,NaN,S
669,670,1,1,"Taylor, Mrs. Elmer Zebley (Juliet Cummins Wright)",female,NaN,1,0,19996,52.000,C126,S
795,796,0,2,"Otter, Mr. Richard",male,39.0,0,0,28213,13.000,NaN,S
416,417,1,2,"Drew, Mrs. James Vivian (Lulu Thorne Christian)",female,34.0,1,1,28220,32.500,NaN,S


Train - Test 셋을 분리하지 않고 하나로 묶어서 전처리를 일부 진행합니다.

분리하지 않고 묶어서 진행하는 이유는 Family를 분류하기 위해서인데 같은 가족 구성원임에도 불구하고 test셋에 그 구성원이 있는 경우가 있습니다. 따라서 하나로 묶은 뒤 일부 전처리를 한번에 하였습니다.

In [17]:
x_train = train.drop(['PassengerId', 'Survived'], axis=1)
x_test = test.drop(['PassengerId'], axis=1)
y_train = train.Survived

all = pd.concat([x_train, x_test]).reset_index(drop=True)

print('[All data] \n', all.isnull().sum())

[All data] 
 Pclass         0
Name           0
Sex            0
Age          263
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin       1014
Embarked       2
dtype: int64


NA값이 존재하는 일부 열에 대해서 전처리를 해줍니다.

Age, Fare의 경우 편차가 크기 때문에 median을 사용하고 4개의 구간으로 나누었습니다.

Cabin의 경우 값이 존재하는 맨 앞 이니셜을 가져왔고, Embarked는 최빈값을 넣었습니다.

In [18]:
age_med = np.nanmedian(all.Age)
fare_med = np.nanmedian(all.Fare)

print(age_med, '//', 
      fare_med, '//',
      all.Cabin.isnull().sum(), '//',
      all.Embarked.isnull().sum(), '\n')

print(all.groupby('Embarked').Embarked.count(), '\n')

# Age median : 28
# Fare median : 14.4542
# Cabin : unknown 1014
# Embarked : two missing values input 'S'

all.Age = all.Age.fillna(age_med)
all.Age = pd.qcut(all.Age, 4, labels=False)
all.Fare = all.Fare.fillna(fare_med)
all.Fare = pd.qcut(all.Fare, 4, labels=False)
all.Cabin = all.Cabin.fillna('unknown')
all.Cabin = all.Cabin.apply(lambda x: x[0])
all.Embarked = all.Embarked.fillna('S')

print(all.isnull().sum())
#print(all.info())

28.0 // 14.4542 // 1014 // 2 

Embarked
C    270
Q    123
S    914
Name: Embarked, dtype: int64 

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64


Name의 경우 다음과 같은 규칙성을 가지고 있습니다.

[Family name, Title. Last name]

이를 family_name과 title으로 분리하여 Family_name과 새로운 성별을 만들어줍니다. Family_name은 가족을 분류하고 가족의 생존률을 계산하기 위해 사용하고, title은 이 사람이 남성인지, 여성인지 혹은 남자 아이인지 분류합니다. 

갑자기 왜 남자아이가 나오지? 라고 생각하실 수 있는데 데이터를 탐색할 때 확인하셨겠지만 title에 'Master'가 존재합니다. 큰 생각 없이 석사학위 소지자라고 생각하였으나, 아무리 시대의 흐름을 고려하여도 10세 미만의 'Master'는 너무나도 이상합니다. 하여 사전을 찾아보니 Mr라고 불릴 정도의 나이가 되지 않은 어린 남자아이를 부르는 호칭이었습니다. 시각화를 통해 파악하신 분들이 있겠지만 이 'Master'에 속한 사람들의 생존률은 일반 성인남성의 생존율보다 훨씬 높게 나타났기 때문에 따로 분류할 필요가 있었습니다.

가족을 Family_name으로 분류한 뒤 가족의 구성원 수를 구하고, 일행이 없는 사람을 분류합니다. 그 후 여성이면서 혼자 탑승한 사람들 중, 다른 사람과 티켓 값이 동일한 사람들이 존재합니다. 이들은 nanny가 아닐까 싶은데 이들 또한 서로 연관성이 있다고 판단하여 'alone'이 아니라 동일한 티켓 값을 가진 사람의 'family_name'을 부여하였습니다. PassengerId가 581번인 여성을 보면 Family_name이 바뀐 것을 확인할 수 있습니다.

Group은 해당 사람이 속한 집단을 더 명확하게 분류하기 위해 [가족-지위-운임-티켓-승선지 ] 로 다시 분류하였습니다. 해당 데이터는 Gender Model에서는 사용하지 않고 아래에서 설명할 blending의 재료를 만들기 위해 XGBClassifier 또는 MLP, 등등의 모델에 사용하기 위해 만들어 주었습니다.

전처리가 끝나고 중복된 항목 및 target과 크게 연관성이 없는 변수들은 제거하고 전처리를 마칩니다.

In [19]:
def data_Preprocessing(df):
    df['Gender'] = df.Name.apply(lambda x: re.split(', |\\. ', x)[1])

    name_man = [df.Gender[df.Sex == 'male'].unique()]
    name_woman = df.Gender[df.Sex == 'female'].unique().tolist()
    name_man = np.delete(name_man, 1).tolist()

    df.loc[df.Gender.isin(name_man), 'Gender'] = 'man'
    df.loc[df.Gender.isin(name_woman), 'Gender'] = 'woman'
    df.loc[df.Gender == 'Master', 'Gender'] = 'boy'

    df['Family_name'] = df.Name.apply(lambda x: x.split(',')[0])
    df.loc[df.Gender == 'man', 'Family_name'] = 'alone'
    df['Family_freq'] = df.groupby('Family_name')['Family_name'].transform('count')
    df.loc[df.Family_freq <= 1, 'Family_name'] = 'alone'

    df.Ticket = df.Ticket.apply(lambda x: x[:-1] + 'X')
    df['Group'] = df.Family_name + '-' + \
                  df.Pclass.map(str) + '-' + \
                  df.Fare.map(str) + '-' + \
                  df.Ticket + '-' + \
                  df.Embarked

    # 581 example
    for i in range(0, len(df)):
        if df.loc[i,'Gender'] != 'man' and df.loc[i,'Family_name'] == 'alone':
            
            df.loc[i,'Family_name'] = \
            df.Family_name[df.Ticket == df.Ticket[i]].iloc[0]

    df = df.drop(['Name', 'Ticket', 'Sex', 'SibSp', 'Parch'], axis=1)

    return df

전처리에서 만들어준 가족 구성원 숫자를 통해 해당 가족의 생존율을 구합니다. 가족 생존율을 구할때는 Train의 Survived를 활용하여 Train-set의 범위에 해당하는 부분만 구한 뒤, 이를 Test-set에도 동일한 가족의 이름을 가진 사람들에게 적용해줍니다. 

Train에서 이 가족의 생존율이 n%만큼 나타났으니 Test에도 n%만큼 나타나지 않을까? 라는 매우 단편적인 예측입니다. 이 단편적인 예측에 몇 가지 **가정**을 더해보았습니다. 

1. 생존율이 존재하지 않는 데이터 중 Pclass로 대표되는 사회경제적 지위가 낮은 사람들은 사망하고, 그 반대의 경우에는 생존했을 것이다. --> **다소 불편하지만 어디까지나 가정입니다.**

2. 여성의 생존율이 높게 나타났기 때문에 여성은 기본적으로 생존했을 것이다.

3. 남자 아이인데 해당 가족 구성원이 모두 생존하였다면 생존했을 것이다.

4. 여성이지만 해당 가족 구성원이 모두 사망하였다면 사망하였을 것이다.

5. 대부분의 남성은 죽었을 것이다.


이 단편적인 가정이 만든 모델은 어떤 결과를 가져올까요


In [20]:
df = data_Preprocessing(all)

df.loc[0:890, 'Survived'] = y_train
temp = df.loc[0:890]
temp['Family_freq'] = temp.groupby('Family_name')['Family_name'].transform('count')
temp['Family_survival'] = \
temp.groupby('Family_name')['Survived'].transform('sum') / \
temp.groupby('Family_name')['Family_name'].transform('count')

temp['Group_freq'] = temp.groupby('Group')['Group'].transform('count')
temp['Group_survival'] = \
temp.groupby('Group')['Survived'].transform('sum') / \
temp.groupby('Group')['Group'].transform('count')


df['Family_survival'] = temp.Family_survival
df['Group_survival'] = temp.Group_survival

del temp

for i in range(891, len(df)):
    df.loc[i, 'Family_survival'] = df.Family_survival[df.Family_name == df.Family_name[i]].iloc[0]
    
df.loc[(df.Family_survival.isnull()) & (df.Pclass == 3), 'Family_survival'] = 0
df.loc[(df.Family_survival.isnull()) & (df.Pclass != 3), 'Family_survival'] = 1

df['predict'] = int(0)
df.loc[df.Gender == 'woman', 'predict'] = 1
df.loc[(df.Gender == 'boy') & (df.Family_survival == 1), 'predict'] = 1
df.loc[(df.Gender == 'woman') & (df.Family_survival == 0), 'predict'] = 0

In [25]:
sub_GM = test[['PassengerId']]
sub_GM['Survived'] = np.int64(df.loc[891:len(df), 'predict'].values)
sub_GM.to_csv('kernel_GM.csv', index=False)

sub_GM.sample(10)

,PassengerId,Survived
161,1053,1
298,1190,0
387,1279,0
142,1034,0
394,1286,0
299,1191,0
77,969,1
250,1142,1
212,1104,0
245,1137,0


In [9]:
sub_GM['Survived'].value_counts()

0    276
1    142
Name: Survived, dtype: int64

![](https://i.imgur.com/nX9GyQR.png)
  
  
이렇게 단순한 가정으로 Public은 0..84688을 달성할 수 있었습니다. 하지만 실제 Private Score와의 격차는 크게 나타났네요. 동일한 자료를 Kaggle Titanic competition에 제출할 경우 0.82xxx정도의 public score를 얻었던 것으로 기억합니다.
 
다음에 시도할 방법은 Blending 입니다. ensembling의 개념으로 본인이 가지고 있는 점수와 유사한 점수대에 있는 다른 결과를 혼합하여 최종 결과를 보정합니다. 내가 가진 예측 결과가 0.82775 일 때, 다른 누군가도 동일한 결과를 나타내고 있는 것을 보신 적이 있을겁니다. 그 사람의 submission과 본인의 submission이 완전히 일치한다고 장담할 수 있을까요?

아래의 예시를 보면 바로 이해가 되실겁니다. 아래의 데이터셋은 각각 Kaggle Titanic의 Kernel에서 가져왔고, 제출한 점수보다 높지 않고 비슷하거나 더 낮은 score를 가지고 있습니다.

In [ ]:
blend = pd.read_csv('../input/titanic-first-blending/blendind_material.csv')
blend.head(5)

best1 ~ best5는 각 submission에서의 Survived를 나타냅니다. 기본적으로 위에서 사용한 sub_GM의 Survived를 베이스로 하여 sub_GM을 포함한 6개의 결과의 합계를 계산합니다.  내가 사망했다고 판단한 누군가를 다른 사람은 생존했다고 판단했을 수 있고, 반대의 경우도 마찬가지입니다. 이를 모두 더해 과반수의 의견에 따라 기존의 판단을 수정하고 최종 결과를 산출합니다.



In [ ]:
for i in range(0, len(blend)):
    for c in range(3, 9):
        blend.loc[i, 'Sum'] = blend.loc[i, 'Sum'] + blend.iloc[i,c]
        
    if blend.loc[i, 'best'] == 0 and blend.loc[i, 'Sum'] > 2.5:
        blend.loc[i, 'Survived'] = 1
            
    if blend.loc[i, 'best'] == 1 and blend.loc[i, 'Sum'] < 2.5:
        blend.loc[i, 'Survived'] = 0

blend.head(10)  

In [ ]:
blend = blend.iloc[:, 0:2]
blend.to_csv('kernel_blend_1st.csv', index=False)

![](https://i.imgur.com/F9q1OC4.png)
Public score는 나아지지 않았지만 Private score는 확실히 개선되었습니다. Private score와 차이가 있는 것은 public socre와 private score를 scoring 할 때 표본수가 다르기 때문에 차이가 있는 것입니다. Private score가 개선되었으니 더 나아졌다고 말할 수 있겠네요.


**Second Blending & Blending Material**

Blending을 하다보면 스스로 다양하게 시도한 blending 결과들을 submission 했을 때 동일한 public score를 받는 경우가 생깁니다. 아래의 자료는 제가 다양하게 blending 시도한 결과들 중 겹치지 않는 5가지 결과를 가지고 다시 blending 하기 위한 작업입니다.

Blending에 필요한 결과들은 어떻게 수집하는지 궁금하신 분들이 있을텐데 그건 

1. 본인의 다양한 모형을 시도한 결과. ( XGB / MLP / SVM / Logistic Regression / etc... )

2. Blending한 결과를 다시 blending material로 사용.

3. Kernel에 감사하게도 공유해주신 유사한 스코어 분들의 결과를 포함.

저의 경우 Gender Model 에 XGBClassifier의 결과를 일부 사용하여 재료를 만들어보기도 했습니다.  Gender Model의 기본 가정이 절대 다수의 남성이 사망하였다는 전제가 있기 때문에 XGBClassifier를 통해 생존했다고 판단한 남성들의 값 만을 대입해보기도 하였습니다. 최소한 3개 이상의 서로 독립인 결과를 가지고 blending해야 성능을 기대할 수 있습니다.

In [ ]:
blend2 = pd.read_csv('../input/titanic-second-blending/blendind_material2.csv')

for i in range(0, len(blend2)):
    for c in range(2,7):
        blend2.iloc[i,7] = blend2.iloc[i,7] + blend2.iloc[i,c]
        
    if blend2.loc[i, 'Survived'] == 0 and blend2.loc[i, 'Sum'] > 2.5:
        blend2.loc[i, 'Survived'] = 1
    if blend2.loc[i, 'Survived'] == 1 and blend2.loc[i, 'Sum'] < 2.5:
        blend2.loc[i, 'Survived'] = 0

blend2.head(10)   

In [ ]:
blend2 = blend2.iloc[:, 0:2]
blend2.to_csv('kernel_blend_2nd.csv', index=False)

![](https://i.imgur.com/MEeNhtS.png)

두 번째 blending한 결과 또한 위와 동일하게 public score는 나아지지 않았으나 private score는 확실히 개선되었습니다. 

제가 90% 이상의 score를 달성하기까지 blending을 하면 할수록 공통적으로 여러 결과에서 헷갈려하는 몇몇 PassengerId들이 있었습니다. 위를 보면 Sum에 5 또는 0이라면 모든 결과에서 이견 없이 생존 혹은 사망이라고 판단한 경우입니다. 하지만 Sum이 1 ~ 4라면 각각의 결과들도 사람과 마찬가지로 생존여부를 헷갈려합니다. 해당 PassengerId들을 보정 하다보면 blending 결과가 나아지지 않는 상황을 극복하실 수 있을거라 생각합니다.

실제 눈으로 확인할 수 있는 것은 public score 일텐데 이건 public score가 계속 나빠지기 때문에 이 자체가 유의미한 결과인지 의심이 들 수 있다고 생각합니다. 저도 Kaggle competition에 blending한 결과를 여러차례 제출하였고, 동일한 결과 혹은 더 나빠진 결과를 받은 경우가 훨씬 많습니다. 하지만 그 과정 역시 눈에 보이지 않을 뿐 실제로는 발전하고 있는 과정이라고 생각합니다. Titanic competition에 대해 현재의 상황에서 어떻게 해야 더 나아질지 돌파구를 찾고 계시는 분들께 조금이나마 도움이 될 수 있었으면 좋겠습니다.

감사합니다.